# EKSTRAKSI PALET WARNA UNTUK KOMPRESI GAMBAR DIGITAL MENGGUNAKAN ALGORITMA K-MEANS

TUGAS AKHIR

Ahmad Ma'ruf  
NIM. 2020150012

Teknik Informatika  
Fakultas Teknik dan Ilmu Komputer  
Universitas Sains Al-Qur'an Jawa Tengah di Wonosobo  
2024

In [1]:
# uncompressing dataset (img file)
!unzip "./source.zip"

Archive:  ./source.zip
   creating: source/
  inflating: source/a1.jpg           
  inflating: source/a2.jpg           
  inflating: source/a3.jpg           
  inflating: source/attribution.csv  
  inflating: source/b1.jpg           
  inflating: source/b2.jpg           
  inflating: source/b3.jpg           
  inflating: source/c1.jpg           
  inflating: source/c2.jpg           
  inflating: source/c3.jpg           
  inflating: source/d1.jpg           
  inflating: source/d2.jpg           
  inflating: source/d3.jpg           


In [2]:
# Import library
import os
from io import BytesIO
from typing import Tuple
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
import datetime
from time import time
import csv

In [3]:
# create function for extracting experiment id
def extract_id(id: str, separator : str):
  # Split the experiment ID into its components
    components = id.split(separator)

    # Extract file name, K value, and experiment number
    file = f"{components[0]}.jpg"
    k = int(components[1])
    p = int(components[2])

    return file, k, p

In [4]:
# load image function. load image file as PIL Object
def load_img(path: str):
  return Image.open(path)

In [5]:
# Color palette generator (K-Means)
def get_color_palette(img: Image, n_color: int) -> Tuple[np.ndarray, KMeans]:
  """
  Extracts a color palette from an image using K-Means clustering.

  Args:
      img: PIL Image object representing the input image.
      n_color: The desired number of colors in the resulting palette.

  Returns:
      A tuple containing:
          - palette (np.ndarray): A NumPy array representing the color palette (RGB values).
          - kmeans (KMeans): The fitted KMeans model object.
  """
  # RESIZE into half
  width, height = img.size
  new_size = (width//2, height//2)
  resized_image = img.resize(new_size)

  # Convert the PIL Image to a NumPy array and normalize pixel values  to range [0, 1]
  img = np.asarray(resized_image) / 255.0

  # Get the image dimensions (height, width, and number of channels)
  h, w, c = img.shape

  # Reshape the image data for K-Means clustering
  """
  This line reshapes the image data into a 2D array. Each row represents a pixel,
  and the columns represent the color channels (e.g., Red, Green, Blue). This format
  is suitable for K-Means clustering, which operates on data points.
  """
  img_arr = img.reshape(h * w, c)

  # Perform K-Means clustering to identify representative colors
  # Multiply by 255 to convert back to original pixel value range (0-255)
  kmeans = KMeans(n_clusters=n_color, n_init="auto").fit(img_arr)

  # Extract the cluster centers (centroids) as the color palette
  palette = np.asarray(kmeans.cluster_centers_ * 255,).astype(int)

  # Return the color palette and the fitted KMeans model
  return palette, kmeans


In [6]:
def quantize_img(img: Image, kmeans: KMeans) -> np.ndarray:
  """
  Quantizes an image using a pre-trained KMeans model.

  Args:
      img: PIL Image object representing the input image.
      kmeans: A fitted KMeans model object used for color quantization.

  Returns:
      A NumPy array representing the quantized image.
  """

  # Convert the PIL Image to a NumPy array and normalize pixel values to range [0, 1]
  img_np = np.asarray(img) / 255.0

  # Get the image dimensions (height, width, and number of channels)
  h, w, c = img_np.shape

  # Reshape the image data into a 2D array for KMeans prediction
  flatten = img_np.reshape(h * w, c)

  # Predict cluster labels for each pixel using the KMeans model
  # The output is an array containing the predicted cluster label (integer)
  pixel_rgb_clusters = kmeans.predict(flatten)

  # Quantize image by assigning cluster centers (centroids) to pixels
  img_quantized = kmeans.cluster_centers_[pixel_rgb_clusters]

  # Reshape the quantized image back to its original dimensions
  return img_quantized.reshape(h, w, c)


In [7]:
def save_quantized_img(quantized_img: np.ndarray, filename: str, resize_factor=0.6):
  """
  Saves a quantized image as a JPG file.

  Args:
      quantized_img: A NumPy array representing the quantized image.
      filename: The desired filename (including '.jpg' extension) to save the image.
  """

  pixel_array_uint8 = (quantized_img * 255).astype(np.uint8)

  # Convert the quantized image back to a PIL Image object
  quantized_image = Image.fromarray(pixel_array_uint8)

  if resize_factor < 1:  # Only resize if a scaling factor is provided (between 0 and 1)
      # Get the original dimensions
      w, h = quantized_image.size

      # Calculate the new dimensions for 75% size
      new_w = int(w * resize_factor)
      new_h = int(h * resize_factor)

      # Resize the image using PIL's resize method
      quantized_image = quantized_image.resize((new_w, new_h), Image.Resampling.LANCZOS)  # Use ANTIALIAS for smoother resizing

  # Save the quantized image as a JPEG
  quantized_image.save(filename, optimize=True, quality=80)

In [8]:
def save_quantized_img(
    quantized_img: np.ndarray,
    filename: str, resize_factor=0.6):
  """
  Saves a quantized image as a JPG file.
  Args:
      quantized_img: A NumPy array of the quantized image.
      filename: The desired filename
      resize_factor: A scaling factor to resize the image.
  """

  pixel_array_uint8 = (quantized_img * 255).astype(np.uint8)

  # Convert the quantized image back to a PIL Image object
  quantized_image = Image.fromarray(pixel_array_uint8)

  if resize_factor < 1:
      # Get the original dimensions
      w, h = quantized_image.size

      # Calculate the new dimensions
      new_w = int(w * resize_factor)
      new_h = int(h * resize_factor)

      # Resize the image using PIL's resize method
      quantized_image = quantized_image.resize(
          (new_w, new_h),
          Image.Resampling.LANCZOS)

  # Save the quantized image as a JPEG
  quantized_image.save(filename, optimize=True, quality=80)

In [9]:
def calculate_mse_psnr_metrics(original_img: np.ndarray, quantized_img: np.ndarray) -> (float, float):
  """
  Calculates Mean Squared Error (MSE) and Peak Signal-to-Noise Ratio (PSNR) between two images.

  Args:
      original_img: A NumPy array representing the original image.
      quantized_img: A NumPy array representing the quantized image.

  Returns:
      A tuple containing:
          - mse (float): The Mean Squared Error between the images.
          - psnr (float): The Peak Signal-to-Noise Ratio between the images.
  """

  # Calculate Mean Squared Error (MSE)
  # Square the difference and compute mean
  mse = np.mean(np.square(original_img - quantized_img))

  # Calculate Peak Signal-to-Noise Ratio (PSNR)
  max_intensity = 255.0
  psnr = 10 * np.log10(max_intensity**2 / mse)

  return mse, psnr


In [10]:
def calculate_mse_psnr_metrics(
    original_img: np.ndarray,
    quantized_img: np.ndarray) -> (float, float):
  """
  Calculates Mean Squared Error (MSE)
  and Peak Signal-to-Noise Ratio (PSNR)
  Args:
      original_img: A NumPy of the original image.
      quantized_img: A NumPy of the quantized image.

  Returns:
      A tuple containing:
          - mse (float):
          - psnr (float):
  """

  # Calculate Mean Squared Error (MSE)
  # Square the difference and compute mean
  mse = np.mean(
      np.square(original_img - quantized_img))

  # Calculate Peak Signal-to-Noise Ratio (PSNR)
  max_intensity = 255.0
  psnr = 10 * np.log10(max_intensity**2 / mse)

  return mse, psnr


In [11]:
def calculate_compression_ratio(original_img_path: str, compressed_img_path: str):
  """
  Calculates the compression ratio achieved by comparing the original and compressed image sizes.

  This function takes the paths to the original and compressed image files and calculates the compression ratio.
  A higher compression ratio indicates that the compressed image is significantly smaller than the original image.

  Args:
      original_img_path: Path to the original image file (e.g., "path/to/image.jpg").
      compressed_img_path: Path to the compressed image file (e.g., "compressed_image.jpg").

  Returns:
      The compression ratio (float) between 0 and 1, where 1 indicates higher compression
      (compressed image size is much smaller than the original).
  """

  original_size = os.path.getsize(original_img_path)
  compressed_size = os.path.getsize(compressed_img_path)

  compression_ratio = 1 - (compressed_size / original_size)

  return compression_ratio, original_size, compressed_size


## Start Experiment

In [12]:
# SOURCE IMG FOLDER
sourcepath = './source'

# CREATE RESULT FOLDER
now = datetime.datetime.now()

# Adjust for GMT+7 assuming your system clock reflects it
offset_hours = 7
now = now + datetime.timedelta(hours=offset_hours)
formatted_datetime = now.strftime("%y%m%d_%H-%M")  # YYMMDD_H-M format

# RESULT PATH
resultpath = os.path.join("./", formatted_datetime)

# Create the folder if it doesn't exist
if not os.path.exists(resultpath):
  os.makedirs(resultpath)


In [13]:
# Nama file gambar yang akan digunakan
files = [
    'a1', 'a2', 'a3', 'b1', 'b2', 'b3',
    'c1', 'c2', 'c3', 'd1', 'd2', 'd3'
]

k_number = [8,16,32,64,96,128] # Jumlah Klaster
exp_count = [1,2,3] # index percobaan

# GENERATE EXPERIMENT IDs
exp_ids = [
    f"{file}_{k}_{cnt}"
    for file in files
    for k in k_number
    for cnt in exp_count
]

print("Jumlah eksperimen: " + str(len(exp_ids)))
print("ID eksperimen :")
print(exp_ids)

Jumlah eksperimen: 216
ID eksperimen :
['a1_8_1', 'a1_8_2', 'a1_8_3', 'a1_16_1', 'a1_16_2', 'a1_16_3', 'a1_32_1', 'a1_32_2', 'a1_32_3', 'a1_64_1', 'a1_64_2', 'a1_64_3', 'a1_96_1', 'a1_96_2', 'a1_96_3', 'a1_128_1', 'a1_128_2', 'a1_128_3', 'a2_8_1', 'a2_8_2', 'a2_8_3', 'a2_16_1', 'a2_16_2', 'a2_16_3', 'a2_32_1', 'a2_32_2', 'a2_32_3', 'a2_64_1', 'a2_64_2', 'a2_64_3', 'a2_96_1', 'a2_96_2', 'a2_96_3', 'a2_128_1', 'a2_128_2', 'a2_128_3', 'a3_8_1', 'a3_8_2', 'a3_8_3', 'a3_16_1', 'a3_16_2', 'a3_16_3', 'a3_32_1', 'a3_32_2', 'a3_32_3', 'a3_64_1', 'a3_64_2', 'a3_64_3', 'a3_96_1', 'a3_96_2', 'a3_96_3', 'a3_128_1', 'a3_128_2', 'a3_128_3', 'b1_8_1', 'b1_8_2', 'b1_8_3', 'b1_16_1', 'b1_16_2', 'b1_16_3', 'b1_32_1', 'b1_32_2', 'b1_32_3', 'b1_64_1', 'b1_64_2', 'b1_64_3', 'b1_96_1', 'b1_96_2', 'b1_96_3', 'b1_128_1', 'b1_128_2', 'b1_128_3', 'b2_8_1', 'b2_8_2', 'b2_8_3', 'b2_16_1', 'b2_16_2', 'b2_16_3', 'b2_32_1', 'b2_32_2', 'b2_32_3', 'b2_64_1', 'b2_64_2', 'b2_64_3', 'b2_96_1', 'b2_96_2', 'b2_96_3', 'b2_12

In [14]:
csv_headers = ["id","img_type", "k",
               "mse", "psnr", "original_size",
               "compressed_size", "cr", "comp_time"]

result = []
count = 1;
start_compute = time();
for exp_id in exp_ids:
  start_time = time()  # Start time measurement

  file, k, p = extract_id(exp_id, "_")

  source_img_path = f"{sourcepath}/{file}"
  result_img_path = f"{resultpath}/{exp_id}.jpg"

  image = load_img(source_img_path)
  palette, k_means = get_color_palette(image, k)
  quantized_img = quantize_img(image, k_means)
  save_quantized_img(quantized_img, result_img_path)

  img_np = np.asarray(image) / 255.0
  mse, psnr = calculate_mse_psnr_metrics(
      img_np, quantized_img)

  compression_ratio, original_size, compressed_size =  calculate_compression_ratio(
      source_img_path, result_img_path)

  computation_time = time() - start_time

  # CONVERT THE VALUE INTO float positional notation
  converted_mse = f"{mse:.7f}"
  converted_psnr = f"{psnr:.2f}"
  converted_compression_ratio = f"{compression_ratio:.2f}"
  converted_computation_time = f"{computation_time:.2f}"

  res = [exp_id, exp_id[0], k,
         converted_mse, converted_psnr,
         original_size, compressed_size,
         converted_compression_ratio,
         converted_computation_time]

  result.append(res)

  print(f"""
  Percobaan ke {count}
    ID : {exp_id}
    MSE: {converted_mse}
    PSNR: {converted_psnr}
    CR: {converted_compression_ratio}
    Time: {converted_computation_time}
  """)
  count += 1

total_compute = time() - start_compute

print(f"total computational time {total_compute/60} minutes")


  Percobaan ke 1
    ID : a1_8_1
    MSE: 0.0034487
    PSNR: 72.75
    CR: 0.47
    Time: 0.32
  

  Percobaan ke 2
    ID : a1_8_2
    MSE: 0.0034506
    PSNR: 72.75
    CR: 0.47
    Time: 0.16
  

  Percobaan ke 3
    ID : a1_8_3
    MSE: 0.0031700
    PSNR: 73.12
    CR: 0.46
    Time: 0.21
  

  Percobaan ke 4
    ID : a1_16_1
    MSE: 0.0014842
    PSNR: 76.42
    CR: 0.46
    Time: 0.22
  

  Percobaan ke 5
    ID : a1_16_2
    MSE: 0.0014836
    PSNR: 76.42
    CR: 0.46
    Time: 0.28
  

  Percobaan ke 6
    ID : a1_16_3
    MSE: 0.0015051
    PSNR: 76.36
    CR: 0.46
    Time: 0.25
  

  Percobaan ke 7
    ID : a1_32_1
    MSE: 0.0007864
    PSNR: 79.17
    CR: 0.47
    Time: 0.41
  

  Percobaan ke 8
    ID : a1_32_2
    MSE: 0.0007817
    PSNR: 79.20
    CR: 0.47
    Time: 0.40
  

  Percobaan ke 9
    ID : a1_32_3
    MSE: 0.0007871
    PSNR: 79.17
    CR: 0.47
    Time: 0.35
  

  Percobaan ke 10
    ID : a1_64_1
    MSE: 0.0004557
    PSNR: 81.54
    CR: 0.47
    Time: 

In [15]:
# print(result)
# SAVE TO CSV

# CREATE RESULT file name
now = datetime.datetime.now()

# Adjust for GMT+7
offset_hours = 7
now = now + datetime.timedelta(hours=offset_hours)

formatted_datetime = now.strftime("%y%m%d_%H-%M")

resultfile = f"{resultpath}/result-{formatted_datetime}.csv"

with open(resultfile, "w", newline="") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(csv_headers)  # Write header row
  writer.writerows(result)  # Write data rows

print("CSV file saved successfully!")

CSV file saved successfully!


In [16]:
# DOWNLOAD THE RESULT
from google.colab import files
import shutil

shutil.make_archive(resultpath, 'zip', resultpath)
files.download(f"{resultpath}.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# uji coba gambar lain diluar dataset. asumsi nilai k optimal = 96

In [17]:
k = 96
test_file = "test-img.jpeg"
start_time = time()  # Start time measurement
image = load_img(test_file)
palette, k_means = get_color_palette(image, k)

quantized_img = quantize_img(image, k_means)
save_quantized_img(quantized_img, "result.jpg", 0.6)

img_np = np.asarray(image) / 255.0
mse, psnr = calculate_mse_psnr_metrics(
    img_np, quantized_img)

compression_ratio, original_size, compressed_size =  calculate_compression_ratio(
    test_file, "result.jpg")
computation_time = time() - start_time
# CONVERT THE VALUE INTO float positional notation
converted_mse = f"{mse:.7f}"
converted_psnr = f"{psnr:.2f}"
converted_compression_ratio = f"{compression_ratio:.2f}"
converted_computation_time = f"{computation_time:.2f}"

print(f"""
  MSE: {converted_mse}
  PSNR: {converted_psnr}
  CR: {converted_compression_ratio}
  TIME : {converted_computation_time}
""")


  MSE: 0.0001542
  PSNR: 86.25
  CR: 0.29
  TIME : 25.42

